# AIS Trajectory

### Importere biblioteker

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.tuner import Tuner
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting import BaseModel, MAE, Baseline


c:\Users\avira\OneDrive - NTNU\Master EMIL\1. semester\ML\TDT4173-Gruppe-8\.venv\Lib\site-packages\pytorch_forecasting\models\base_model.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# Load data
ais_evaluations = pd.read_csv('data/ais_test.csv')
ais_train = pd.read_csv('data/ais_train.csv', sep='|')

# Preprocess data
ais_train['datetime'] = pd.to_datetime(ais_train['time'])
ais_evaluations['datetime'] = pd.to_datetime(ais_evaluations['time'])


# Dropp the vessels that appears less than 100 times 
vessel_counts = ais_train['vesselId'].value_counts().reset_index()
vessel_counts.columns = ['vesselId', 'count']
vessel_counts = vessel_counts[vessel_counts['count'] > 100]


# print all vessels and their counts
display(vessel_counts)


# Convert time to integer
ais_train = ais_train.sort_values("datetime")
ais_train['time_idx'] = (ais_train['datetime'] - ais_train['datetime'].min()).dt.seconds

ais_evaluations = ais_evaluations.sort_values("datetime")
ais_evaluations['time_idx'] = (ais_evaluations['datetime'] - ais_train['datetime'].min()).dt.seconds

display(ais_train.head())

# Create TimeSeriesDataSet
max_prediction_length = int(ais_train['time_idx'].max() * 0.1)
max_encoder_length = int(ais_train['time_idx'].max() * 0.1)
training_cutoff = ais_train['time_idx'].max() - max_prediction_length


training = TimeSeriesDataSet(
    ais_train[lambda x: x.time_idx <= training_cutoff],
    time_idx='time_idx',
    target=['longitude', 'latitude'],
    group_ids=['vesselId'],
    min_encoder_length=max_encoder_length//2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    allow_missing_timesteps=True,
    time_varying_unknown_reals=[
        "cog",
        "sog",
        "rot",
        "heading",
    ],
)



,vesselId,count
0,6323f2287abc89c0a9631e57,8656
1,61e9f466b937134a3c4c0273,8626
2,61e9f464b937134a3c4c025f,7958
3,61e9f465b937134a3c4c026d,7928
4,61e9f465b937134a3c4c0271,7689
...,...,...
681,61e9f45eb937134a3c4c0235,250
682,61e9f39ab937134a3c4bfdb9,197
683,61e9f45cb937134a3c4c022b,196
684,61e9f42cb937134a3c4c00f9,191


,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,datetime,time_idx
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,2024-01-01 00:00:25,0
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,2024-01-01 00:00:36,11
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,2024-01-01 00:01:45,80
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,2024-01-01 00:03:11,166
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,2024-01-01 00:03:51,206


c:\Users\avira\OneDrive - NTNU\Master EMIL\1. semester\ML\TDT4173-Gruppe-8\.venv\Lib\site-packages\pytorch_forecasting\data\timeseries.py:1282: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__vesselId': '61e9f3cbb937134a3c4bff09'}]
  warnings.warn(


In [4]:
# create validation and training dataset
validation = TimeSeriesDataSet.from_dataset(training, ais_train, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 32
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

# create dataloaders for model
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

c:\Users\avira\OneDrive - NTNU\Master EMIL\1. semester\ML\TDT4173-Gruppe-8\.venv\Lib\site-packages\pytorch_forecasting\data\timeseries.py:1282: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__vesselId': '61e9f3adb937134a3c4bfe37'}]
  warnings.warn(


: 

In [ ]:
# create dataloaders for model
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*2, num_workers=0)

# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)

# Print the size of predictions and ground truth
print(f"Predictions shape: {baseline_predictions.output.shape}")
print(f"Ground truth shape: {baseline_predictions.y.shape}")

MAE()(baseline_predictions.output, baseline_predictions.y)


C:\Users\avira\AppData\Local\Temp\ipykernel_42348\3048310298.py:7: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of dependencies in default parameter settings. Users who wish to ensure their code continues using 'ranger' as optimizer should ensure that pytorch_optimizer is installed, and set the optimizer parameter explicitly to 'ranger'.
  baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
c:\Users\avira\OneDrive - NTNU\Master EMIL\1. semester\ML\TDT4173-Gruppe-8\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hy

c:\Users\avira\OneDrive - NTNU\Master EMIL\1. semester\ML\TDT4173-Gruppe-8\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="cpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    loss=QuantileLoss(),
    optimizer="Ranger"
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=50,
    accelerator="cpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=50,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    optimizer="Ranger",
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=200,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte mean absolute error on validation set
predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))
MAE()(predictions.output, predictions.y)

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(raw_predictions.x, raw_predictions.output, idx=idx, add_loss_to_title=True)